# Load Dataset

## Prework to Generate Dataset CSVs

In [ ]:
!pip install -U scikit-learn

In [12]:
import urllib.request
import json
import csv
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import pickle

In [2]:
with open('/mnt/c/Users/willi/Downloads/intents.json') as json_data:
    intents = json.load(json_data)

In [3]:
print(intents)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi there', 'How are you', 'Is anyone there?', 'Hello', 'Good day'], 'responses': ['Hello, thanks for asking', 'Good to see you again', 'Hi there, how can I help?'], 'context': ['']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye', 'Nice chatting to you, bye', 'Till next time'], 'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'], 'context': ['']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", 'Awesome, thanks', 'Thanks for helping me'], 'responses': ['Happy to help!', 'Any time!', 'My pleasure'], 'context': ['']}, {'tag': 'noanswer', 'patterns': [], 'responses': ["Sorry, can't understand you", 'Please give me more info', 'Not sure I understand'], 'context': ['']}, {'tag': 'options', 'patterns': ['How you could help me?', 'What you can do?', 'What help you provide?', 'How you can be helpful?', 'What support is offered'], 'responses': ['I can guide you through Adverse drug r

In [4]:
temp_intent_dict = {}
temp_utterance = []
temp_intents = []
for i in intents['intents']:
    for j in i['patterns']:
        temp_intents.append(str(i['tag']))
        temp_utterance.append(str(j))        
temp_intent_dict['utterance'] = temp_utterance
temp_intent_dict['intent'] = temp_intents

#for intent in intents['intents']:
#    temp_intents[f"{intent['tag']}"] = intent['patterns']

In [5]:
print(temp_intent_dict)

{'utterance': ['Hi there', 'How are you', 'Is anyone there?', 'Hello', 'Good day', 'Bye', 'See you later', 'Goodbye', 'Nice chatting to you, bye', 'Till next time', 'Thanks', 'Thank you', "That's helpful", 'Awesome, thanks', 'Thanks for helping me', 'How you could help me?', 'What you can do?', 'What help you provide?', 'How you can be helpful?', 'What support is offered', 'How to check Adverse drug reaction?', 'Open adverse drugs module', 'Give me a list of drugs causing adverse behavior', 'List all drugs suitable for patient with adverse reaction', 'Which drugs dont have adverse reaction?', 'Open blood pressure module', 'Task related to blood pressure', 'Blood pressure data entry', 'I want to log blood pressure results', 'Blood pressure data management', 'I want to search for blood pressure result history', 'Blood pressure for patient', 'Load patient blood pressure result', 'Show blood pressure results for patient', 'Find blood pressure results by ID', 'Find me a pharmacy', 'Find pha

In [6]:
temp_intent_df = pd.DataFrame.from_dict(temp_intent_dict)

In [7]:
temp_intent_df.head(10)

,utterance,intent
0,Hi there,greeting
1,How are you,greeting
2,Is anyone there?,greeting
3,Hello,greeting
4,Good day,greeting
5,Bye,goodbye
6,See you later,goodbye
7,Goodbye,goodbye
8,"Nice chatting to you, bye",goodbye
9,Till next time,goodbye


In [8]:
temp_intent_df['utterance'] = temp_intent_df['utterance'].apply(lambda x: "\"" + str(x) + "\"")

In [9]:
temp_intent_df.head(10)

,utterance,intent
0,"""Hi there""",greeting
1,"""How are you""",greeting
2,"""Is anyone there?""",greeting
3,"""Hello""",greeting
4,"""Good day""",greeting
5,"""Bye""",goodbye
6,"""See you later""",goodbye
7,"""Goodbye""",goodbye
8,"""Nice chatting to you, bye""",goodbye
9,"""Till next time""",goodbye


In [10]:
intent_new_df = pd.read_csv('/mnt/c/Users/willi/Downloads/intent_dataset_new.csv')

In [13]:
intent_new_df.head(10)

,utterance,intent
0,"""Hello!""",greeting
1,"""Hi there!""",greeting
2,"""Hey!""",greeting
3,"""Good morning!""",greeting
4,"""Good afternoon!""",greeting
5,"""Good evening!""",greeting
6,"""Howdy!""",greeting
7,"""Greetings!""",greeting
8,"""What's up?""",greeting
9,"""How's it going?""",greeting


In [14]:
intent_full_df = pd.concat([temp_intent_df,intent_new_df],axis=0)

In [15]:
intent_full_df.head(10)

,utterance,intent
0,"""Hi there""",greeting
1,"""How are you""",greeting
2,"""Is anyone there?""",greeting
3,"""Hello""",greeting
4,"""Good day""",greeting
5,"""Bye""",goodbye
6,"""See you later""",goodbye
7,"""Goodbye""",goodbye
8,"""Nice chatting to you, bye""",goodbye
9,"""Till next time""",goodbye


In [16]:
#temp_intent_df.shape
#intent_raw_df.shape
intent_full_df.shape

(265, 2)

In [17]:
intent_full_df.to_csv('/mnt/c/Users/willi/Downloads/intent_full_dataset.csv',index=False)

In [65]:
temp_response_dict = {}
temp_responses = []
temp_intents = []
for i in intents['intents']:
    for j in i['responses']:
        temp_intents.append(str(i['tag']))
        temp_responses.append(str(j))        
temp_response_dict['responses'] = temp_responses
temp_response_dict['intent'] = temp_intents

In [68]:
temp_response_df = pd.DataFrame.from_dict(temp_response_dict)
temp_response_df.head(10)

,responses,intent
0,"Hello, thanks for asking",greeting
1,Good to see you again,greeting
2,"Hi there, how can I help?",greeting
3,See you!,goodbye
4,Have a nice day,goodbye
5,Bye! Come back again soon.,goodbye
6,Happy to help!,thanks
7,Any time!,thanks
8,My pleasure,thanks
9,"Sorry, can't understand you",noanswer


In [69]:
temp_response_df.to_csv('/mnt/c/Users/willi/Downloads/responses_dataset.csv')

# Augment the Dataset

In [18]:
!pip install textattack
!pip install googletrans

  Obtaining dependency information for textattack from https://files.pythonhosted.org/packages/6a/c9/041fe7d041cd051898fca4000a3f12cca57f85d6372307f8c04e062eee33/textattack-0.3.10-py3-none-any.whl.metadata
  Obtaining dependency information for bert-score>=0.3.5 from https://files.pythonhosted.org/packages/c6/8c/bc5457de4c004b1a623b31f7bc8d0375fb699b7d67df11879098b4b7b7c8/bert_score-0.3.13-py3-none-any.whl.metadata
  Obtaining dependency information for editdistance from https://files.pythonhosted.org/packages/96/8a/db0fd79e8ddb9b5f86f274107c5d0a27ec4f2af88877df1f26c2c6d150cc/editdistance-0.8.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for flair from https://files.pythonhosted.org/packages/af/16/536683088c7306bc51cc3cc58605759ebd83b3f7ffd05a9399f4b99c8614/flair-0.13.1-py3-none-any.whl.metadata
  Obtaining dependency information for language-tool-python from https://files.pythonhosted.org/packages/d2/82/feb4f01e269267021f1e3a3

In [21]:
from textattack.augmentation import EasyDataAugmenter, CharSwapAugmenter
import googletrans
from googletrans import Translator

In [40]:
augmented_intents = {'utterance':[],'intent':[]}
easy_aug = EmbeddingAugmenter()
swap_aug = CharSwapAugmenter()
for intent in intent_full_df['intent'].unique():
    temp_df = intent_full_df.loc[intent_full_df['intent'] == intent]
    for utterance in temp_df['utterance']:
        augmented_intents['utterance'].extend([easy_aug.augment(utterance)[0],swap_aug.augment(utterance)[0]])
        augmented_intents['intent'].extend([intent,intent])
intent_augmented_df = pd.DataFrame.from_dict(augmented_intents)
intent_augmented_df.head(10)

,utterance,intent
0,"""Cuckoo there""",greeting
1,"""HQi there""",greeting
2,"""Mode are you""",greeting
3,"""Hw are you""",greeting
4,"""Is person there?""",greeting
5,"""I anyone there?""",greeting
6,"""Howdy""",greeting
7,"""eHllo""",greeting
8,"""Alright day""",greeting
9,"""Gotd day""",greeting


In [41]:
intent_augmented_df.shape

(530, 2)

In [42]:
intent_augmented_full_df = pd.concat([intent_full_df,intent_augmented_df],axis=0)
intent_augmented_full_df.shape

(795, 2)

In [43]:
intent_augmented_full_df.to_csv('/mnt/c/Users/willi/Downloads/intent_augmented_dataset.csv',index=False)

## Dataset Load and Split

In [13]:
intent_df = pd.read_csv('/mnt/c/Users/willi/Downloads/intent_augmented_dataset.csv')

In [14]:
intent_df.head(10)

,utterance,intent
0,"""Hi there""",greeting
1,"""How are you""",greeting
2,"""Is anyone there?""",greeting
3,"""Hello""",greeting
4,"""Good day""",greeting
5,"""Bye""",goodbye
6,"""See you later""",goodbye
7,"""Goodbye""",goodbye
8,"""Nice chatting to you, bye""",goodbye
9,"""Till next time""",goodbye


In [15]:
intent_df.shape

(795, 2)

In [16]:
X = intent_df['utterance']
y = intent_df['intent']
X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(X, y, random_state=42, test_size=0.2, shuffle=True)

In [17]:
test_df = pd.concat([X_test_df,y_test_df],axis=1)
train_df = pd.concat([X_train_df,y_train_df],axis=1)

In [18]:
test_df.head(10)

,utterance,intent
691,"""Could you get my prescription poised for me?""",medication
662,"""Search for hospitals with urgen care services.""",hospital_search
63,"""Hey, good to see you!""",greeting
531,"""Whereof are my blood pressure levels?""",blood_pressure
66,"""Bye!""",goodbye
749,"""I require the attention of a physician.""",doctor
382,"""'m grateful.""",thanks
683,"""I'd like to order a new batch of my medicatio...",medication
554,"""WhaW's the verdict on my blood pressure today?""",blood_pressure
624,"""Where can I byu over-the-counter medications?""",pharmacy_search


In [19]:
train_df.head(10)

,utterance,intent
360,"""oodnight!""",goodbye
264,"""I have digestive issues such as bloating or i...",symptoms
440,"""Rhat do you specialize in?""",options
328,"""yBe!""",goodbye
486,"""What are the riIks or drawbacks of this medic...",adverse_drug
368,"""Thaxnk you""",thanks
79,"""See you soon!""",goodbye
148,"""What are my blood pressure levels?""",blood_pressure
529,"""How is my blood pressure looking yesterday?""",blood_pressure
381,"""I'm appreciate.""",thanks


# RAG Intent Identification

In [1]:
!pip install -U torch
!pip install -U transformers
!pip install -qU "semantic-router[local]"

In [7]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from semantic_router import Route
from semantic_router import RouteLayer
from semantic_router.encoders import HuggingFaceEncoder

In [51]:
torch.set_default_device("cuda")

#model = AutoModelForCausalLM.from_pretrained('microsoft/phi-1_5', torch_dtype='auto')
#tokenizer = AutoTokenizer.from_pretrained('microsoft/phi-1_5')


#model_id = "failspy/Phi-3-mini-128k-instruct-abliterated-v3-GGUF"
#filename = "Phi-3-mini-128k-instruct-abliterated-v3_q8.gguf"

#model = AutoModelForCausalLM.from_pretrained(model_id, gguf_file=filename, torch_dtype='auto')
#tokenizer = AutoTokenizer.from_pretrained(model_id, gguf_file=filename)

In [20]:
#adverse_drug_route = Route(
#    name= "adverse_drug",
#    utterances= ["How to check Adverse drug reaction?", "Open adverse drugs module", "Give me a list of drugs causing adverse behavior", "List all drugs suitable for patient with adverse reaction", "Which drugs dont have adverse reaction?" ],
#    responses= ["Navigating to Adverse drug reaction module"],
#)
#blood_pressure_route = Route(
#    name= "blood_pressure",
#    utterances= ["Open blood pressure module", "Task related to blood pressure", "Blood pressure data entry", "I want to log blood pressure results", "Blood pressure data management" ],
#    responses= ["Navigating to Blood Pressure module"],
#)

routes=[]
intents = train_df['intent'].unique()

for intent in intents:
    temp_df = train_df.loc[train_df['intent'] == intent]
    temp_utterances = temp_df['utterance'].tolist()
    intent_name = str(intent)
    intent_name = Route(
        name=intent,
        utterances=temp_utterances,
        responses=['N/A']
    )
    routes.append(intent_name)

print(routes)

#response_dict = {'blood_pressure': {'name': 'blood_pressure', 'responses': ["Navigating to Blood Pressure module"]}}

#routes=[adverse_drug_route,blood_pressure_route]

[Route(name='goodbye', utterances=['"oodnight!"', '"yBe!"', '"See you soon!"', '"Farewell"', '"Placid out!"', '"Take care!"', '"Farewell!"', '"Goodnight!"', '"See you later"', '"Until next time!"', '"ByO"', '"Behold ya!"', '"Bye-Uye!"', '"Take it effortless!"', '"See you almost!"', '"Take healthcare!"', '"Have a good one!"', '"Hiya!"', '"Have a ood one!"', '"I must be going."', '"Have a great dgy!"', '"Take caNre!"', '"Gntil next time!"', '"Till next itme"', '"Till next time"', '"Catch you ater!"', '"Have a buena one!"', '"So long!"', '"See ya!"', '"See you late"', '"Bye-bye!"', '"Enjoyable chatting to you, bye"', '"See you speedily!"', '"Goodbey"', '"SeCe you soon!"', '"Until future time!"', '"Goodnight!"', '"Nice hatting to you, bye"', '"Bye"', '"Peaec out!"', '"Nice chatting to you, bye"', '"zAdios!"', '"Adieu!"', '"See you afterward"', '"eSe you around!"', '"Adieu!"', '"Ha a good one!"', '"Take it easy!"', '"Catch you subsequent!"', '"So lang!"', '"Goodbye"', '"I should be going."'

In [21]:
encoder = HuggingFaceEncoder()

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

In [26]:
rl = RouteLayer(encoder=encoder, routes=routes)

In [27]:
rl("Open adverse drugs module")

RouteChoice(name='adverse_drug', function_call=None, similarity_score=None)

In [25]:
rl("can you search for my blood pressure please").name

'blood_pressure_search'

In [38]:
rl.to_json("rag_model.json")

2024-07-14 17:44:42 INFO semantic_router.utils.logger Saving route config to rag_model.json


# NN Intent Identification

In [3]:
!pip install -U nltk
!pip install -U flask
!pip install -U flask_cors
!pip install -U keras
!pip install -U tensorflow
# Note that, currently, this is stolen wholesale from https://github.com/katanaml/katana-assistant/blob/master/mlmodels/katana-assistant-model.ipynb
# It will be modified later to suit our purposes, I just want to see if I can get what they have to work!pip install tensorflow

  Obtaining dependency information for flask from https://files.pythonhosted.org/packages/61/80/ffe1da13ad9300f87c93af113edd0638c75138c42a0994becfacac078c06/flask-3.0.3-py3-none-any.whl.metadata
  Obtaining dependency information for Werkzeug>=3.0.0 from https://files.pythonhosted.org/packages/9d/6e/e792999e816d19d7fcbfa94c730936750036d65656a76a5a688b57a656c4/werkzeug-3.0.3-py3-none-any.whl.metadata
  Obtaining dependency information for itsdangerous>=2.1.2 from https://files.pythonhosted.org/packages/04/96/92447566d16df59b2a776c0fb82dbc4d9e07cd95062562af01e408583fc4/itsdangerous-2.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for click>=8.1.3 from https://files.pythonhosted.org/packages/00/2e/d53fa4befbf2cfa713304affc7ca780ce4fc1fd8710527771b58311a3229/click-8.1.7-py3-none-any.whl.metadata
  Obtaining dependency information for blinker>=1.6.2 from https://files.pythonhosted.org/packages/bb/2a/10164ed1f31196a2f7f3799368a821765c62851ead0e630ab52b8e14b4d0/blinker-1.8.2

In [2]:
# Note that, currently, this is stolen wholesale from https://github.com/katanaml/katana-assistant/blob/master/mlmodels/katana-assistant-model.ipynb
# It will be modified later to suit our purposes, I just want to see if I can get what they have to work

import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD # NOTE: this was originally written with an older version of SGD, so I'm using the legacy version just to implement what they had as they had it
import pandas as pd
import pickle
import random
from flask import Flask, jsonify, request
from flask_cors import CORS, cross_origin

2024-07-14 14:25:43.788191: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-14 14:25:43.811799: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-14 14:25:44.254160: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/mist861/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
words = []
classes = []
documents = []
ignore_words = ['?']
intents = train_df['intent'].unique()
# loop through each sentence in our intents patterns
for intent in intents:
    temp_df = train_df.loc[train_df['intent'] == intent]
    temp_utterances = temp_df['utterance'].tolist()
    for pattern in temp_utterances:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent))
        # add to our classes list
        if intent not in classes:
            classes.append(intent)

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique stemmed words", words)

636 documents
12 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'doctor', 'goodbye', 'greeting', 'hospital_search', 'medication', 'options', 'pharmacy_search', 'symptoms', 'thanks']
659 unique stemmed words ['!', "''", "'d", "'im", "'m", "'re", "'s", "'ve", '(', ')', ',', '.', '24-hour', '24/7', '[', ']', '``', 'a', 'abdomin', 'abils', 'abl', 'about', 'acceiv', 'acecpt', 'ach', 'adieu', 'adv', 'advers', 'afternoon', 'afterward', 'ahc', 'aid', 'ail', 'ajound', 'al', 'almost', 'alright', 'amay', 'an', 'and', 'anticip', 'any', 'anyon', 'aound', 'appoint', 'appreciato', 'apprecy', 'ar', 'area', 'around', 'arrang', 'arrn', 'as', 'assist', 'assocy', 'at', 'attain', 'audit', 'avail', 'aw', 'awesom', 'awesonm', 'barrio', 'batch', 'bbp', 'be', 'been', 'behavy', 'behold', 'bher', 'bil', 'blo', 'bloatx', 'blocod', 'blod', 'bloo', 'blood', 'book', 'booy', 'bp', 'bprescription', 'brac', 'brea', 'bring', 'britng', 'brood', 'brows', 'buen', 'bunch', 'buy', 'by', 'bye', 'bye-by', 

In [15]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word - create base word, in attempt to represent related words
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
#training = np.array(training) # This doesn't work with our version of numpy, boo, it doesn't like the 2d array having arrays of different sizes

# create train and test lists. X - patterns, Y - intents
#train_x = list(training[:,0]) # Same as above, doesn't work
#train_y = list(training[:,1]) # Save as above, doesn't work

train_x, train_y = zip(*training) # So instead of converting to a np array and slicing, we'll split the list then
train_x = np.array(train_x) # Convert the lists, separately, into arrays
train_y = np.array(train_y)

In [16]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

/home/mist861/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-07-14 14:26:27.361708: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-14 14:26:27.361877: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [17]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True) # I had to tweak this slightly, because even using the legacy SGD this was somehow too new?
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

/home/mist861/anaconda3/lib/python3.11/site-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [18]:
# Fit the model
model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)

Epoch 1/200
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 778us/step - accuracy: 0.1619 - loss: 2.3697
Epoch 2/200
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 694us/step - accuracy: 0.4696 - loss: 1.4884
Epoch 3/200
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 632us/step - accuracy: 0.6107 - loss: 0.9823
Epoch 4/200
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 625us/step - accuracy: 0.7707 - loss: 0.6744
Epoch 5/200
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 620us/step - accuracy: 0.7984 - loss: 0.5235
Epoch 6/200
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 624us/step - accuracy: 0.8183 - loss: 0.5042
Epoch 7/200
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 615us/step - accuracy: 0.8195 - loss: 0.4416
Epoch 8/200
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 624us/step - accuracy: 0.8860 - loss: 0.3202
Epoch 9/200
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 628us/step - accuracy: 0.8527 - loss: 0.3722
Epoch 10/200
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 623us/step - accuracy: 0.8624 - loss: 0.3515
Epoch 11/200
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 613us/step - accuracy: 0.8886 - loss: 0.3093
Epoch 12/200
128/12

In [19]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [20]:
def classify_local(sentence):
    ERROR_THRESHOLD = 0.25

    # generate probabilities from the model
    input_data = pd.DataFrame([bow(sentence, words)], dtype=float, index=['input'])
    results = model.predict([input_data])[0]
    # filter out predictions below a threshold, and provide intent index
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], str(r[1])))
    # return tuple of intent and probability

    return return_list

In [41]:
results = classify_local("asdfasdfsadfsadfdsaf")
if float(results[0][1]) < 0.6:
    print('noanswer')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
noanswer


In [41]:
response = classify_local("Please help me find my medication")

found in bag: pleas
found in bag: help
found in bag: me
found in bag: find
found in bag: my
found in bag: med
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


In [44]:
print(response[0][0])

medication


In [45]:
responses_df = pd.read_csv('/mnt/c/Users/willi/Downloads/responses_dataset_augmented.csv')

In [59]:
responses_df.responses[responses_df['intent'] == response[0][0]].item()

'"Navigating to the Medication module..."'

In [38]:
model.save("nn_model.keras")

In [40]:
with open('classes.pickle', 'wb') as file:
    pickle.dump(classes, file)
with open('words.pickle', 'wb') as file:
    pickle.dump(words, file)

# Comparing NN and RAG Intent Identification

In [15]:
rag_results = []
for utterance in test_df['utterance']:
    result = rl(utterance).name
    if result == None:
        result = 'noanswer'
    rag_results.append(result)

print(rag_results)
    
#    result = rl(utterance).name
#    rag_results.append(result)

['medication', 'hospital_search', 'greeting', 'blood_pressure', 'goodbye', 'doctor', 'thanks', 'medication', 'blood_pressure', 'pharmacy_search', 'goodbye', 'greeting', 'hospital_search', 'goodbye', 'greeting', 'thanks', 'doctor', 'hospital_search', 'doctor', 'hospital_search', 'thanks', 'blood_pressure', 'pharmacy_search', 'doctor', 'hospital_search', 'blood_pressure', 'symptoms', 'greeting', 'pharmacy_search', 'symptoms', 'medication', 'greeting', 'blood_pressure', 'hospital_search', 'pharmacy_search', 'adverse_drug', 'hospital_search', 'greeting', 'symptoms', 'noanswer', 'adverse_drug', 'symptoms', 'pharmacy_search', 'medication', 'pharmacy_search', 'blood_pressure', 'options', 'options', 'hospital_search', 'options', 'noanswer', 'options', 'symptoms', 'pharmacy_search', 'goodbye', 'thanks', 'greeting', 'doctor', 'hospital_search', 'pharmacy_search', 'adverse_drug', 'adverse_drug', 'hospital_search', 'adverse_drug', 'doctor', 'blood_pressure', 'greeting', 'blood_pressure_search', 'g

In [34]:
nn_results = []
for utterance in test_df['utterance']:
    result = classify_local(utterance)[0][0]
    nn_results.append(result)

print(nn_results)

found in bag: ``
found in bag: could
found in bag: you
found in bag: get
found in bag: my
found in bag: prescrib
found in bag: for
found in bag: me
found in bag: ''
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
found in bag: ``
found in bag: search
found in bag: for
found in bag: hospit
found in bag: with
found in bag: urg
found in bag: car
found in bag: serv
found in bag: .
found in bag: ''
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
found in bag: ``
found in bag: hey
found in bag: ,
found in bag: good
found in bag: to
found in bag: see
found in bag: you
found in bag: !
found in bag: ''
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
found in bag: ``
found in bag: whereof
found in bag: ar
found in bag: my
found in bag: blood
found in bag: press
found in bag: level
found in bag: ''
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
found in bag: ``
found in bag: bye
found in bag: !
found in bag: ''
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
found in bag: ``
found in bag: i
found in bag: requir
found in bag: the
found in bag: a

In [16]:
from sklearn.metrics import (accuracy_score, precision_score, recall_score,  silhouette_score, multilabel_confusion_matrix as mcm) # Importing sklearn metrics for supervised model evaluation

In [36]:
nn_accuracy = accuracy_score(test_df['intent'], nn_results)
nn_recall = recall_score(test_df['intent'], nn_results, average='macro')
nn_precision = precision_score(test_df['intent'], nn_results, average='macro')
print(f'NN Accuracy: {nn_accuracy}, NN Recall: {nn_recall}, NN Precision: {nn_precision}')

NN Accuracy: 0.9433962264150944, NN Recall: 0.9580327080327079, NN Precision: 0.9561546840958606


In [18]:
rag_accuracy = accuracy_score(test_df['intent'], rag_results)
rag_recall = recall_score(test_df['intent'], rag_results, average='macro')
rag_precision = precision_score(test_df['intent'], rag_results, average='macro')
print(f'RAG Accuracy: {rag_accuracy}, RAG Recall: {rag_recall}, RAG Precision: {rag_precision}')

RAG Accuracy: 0.9119496855345912, RAG Recall: 0.8432930317545703, RAG Precision: 0.8747517069736647


/home/mist861/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# RAG-LLM Integration

In [3]:
! CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python
!pip install ollama

  Obtaining dependency information for ollama from https://files.pythonhosted.org/packages/7d/b7/8cc05807bfbc5b92da7fb94c525e1e56572a08eea7cdf3656e6c5dc6f9b1/ollama-0.2.1-py3-none-any.whl.metadata


In [4]:
import ollama

In [6]:
response = ollama.chat(model='superdrew100/kappa-3-phi-3-4k-instruct-abliterated', messages = [ 
    {"role": "system", "content": "You are a helpful medical assistant. Your response should be similar to: I can do that!  Navigating to the Doctor Search module..."}, 
    {"role": "user", "content": "Can you help me find a doctor?"}
])
print(response['message']['content'])

 Absolutely, I'd be happy to assist you in finding a suitable doctor! To begin with, there are several ways we can approach this task. We can consider factors such as your location, specialty requirements (if any), and preferred healthcare provider types (like family physicians or specialists). 

I will navigate to the Doctor Search module for us. Let me gather some information based on these preferences:

1. Please provide me with your geographical location – city/town, state, or region would be helpful.
2. Do you have any specific medical concerns that require a specialist? If not, we can explore general practitioners in your area.
3. Is there a particular type of healthcare provider you prefer, such as family physicians, internists, or another field?

Once I have these details, I'll be able to help you search for an appropriate doctor nearby.


In [33]:
responses_df = intent_new_df = pd.read_csv('/mnt/c/Users/willi/Downloads/responses_dataset_augmented.csv')

In [34]:
responses_df

,responses,intent
0,Hello! How can I help?,greeting
1,Have a nice day!,goodbye
2,"You're welcome, I'm happy to help!",thanks
3,"I'm sorry, I can't understand you. Please ref...",noanswer
4,"I can assist you with identifying symptoms, fi...",options
5,"Sure, I can help with that! Navigating to the...",adverse_drug
6,"Certainly, I can do that! Navigating to the B...",blood_pressure
7,Of course! Navigating to the Pharmacy Search ...,pharmacy_search
8,Can do! Navigating to the Hospital Search mod...,hospital_search
9,Yep! Navigating to the Blood Pressure Search ...,blood_pressure_search


In [45]:
def generate_rag_respose(prompt):
    intent = rl(prompt).name
    if intent != None:
        response = responses_df.responses[responses_df['intent'] == intent].item()
    else:
        response = responses_df.responses[responses_df['intent'] == 'noanswer'].item()
    msg = ollama.chat(model='superdrew100/kappa-3-phi-3-4k-instruct-abliterated', messages = [ 
        {"role": "system", "content": f"You are a helpful medical assistant. Your response should be similar to: {response}"}, 
        {"role": "user", "content": prompt}
    ])
    return msg['message']['content']

In [50]:
response = generate_rag_respose("Can you help me find my blood pressure?")
print(response)

 Absolutely, I'd be happy to assist you with finding your blood pressure reading. While I don't have direct access to monitor vital signs, there are several ways for you to measure your own blood pressure:

1. Purchase a home blood pressure monitoring device: There are many affordable and accurate personal blood pressure monitors available in the market that can be used at home for regular checks. These devices usually include an inflatable cuff connected to a display unit with measurements shown on it.

2. Visit a healthcare professional: Your doctor or nurse will have access to medical-grade blood pressure measurement equipment, and they can measure your blood pressure during an appointment. Remember that visiting your provider allows for the opportunity to discuss any concerns about high blood pressure or other issues related to your overall health.

3. Use a smartphone app: There are some apps available on mobile devices that help you track and monitor your blood pressure by using 